In [1]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from keras import backend as K
import argparse
import os
from sentence_embeddings import Sentence_Embedder

# Ridle (with and without Language Model)
In this notebook, the performance of "Ridle" is tested on different datasets. The neural network for further testing wasn't changed in any way, except giving it different inputs. Firstly the model is tested by itself to see how it performs in the normal case of just the representation from the RBM as inputs. Then we add our sentence embedding vectors to the representation vector and concatenate them to receive a new bigger vector with the size of input (50) and sentence embedding (384). Finally the whole program is also tested with only the sentence embeddings as inputs, to see how well the program can classify just by getting information from the summary embeddings.

In [2]:
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [2]:
Datasets = ["Books_DBpedia", "ChemicalCompounds_DBpedia", "Company_DBpedia", "DBp_2016-04", 
            "Person_DBpedia", "Songs_DBpedia", "umls", "Universities_DBpedia"]
df1 = pd.DataFrame(Datasets, index=Datasets)
df1["Ridle misclassified"] = ""
df1["Sbert misclassified"] = ""
df1["Misclassified in both"] = ""
df1["ID list Bert"] = ""
df1["Prediction Bert"] = ""
df1["Target Bert"] = ""
df1["ID list Ridle"] = ""
df1["Prediction Ridle"] =""
df1["Target Ridle"] = ""

In [2]:
# GELU Activation function
def gelu(x):
    return 0.5 * x * (1 + K.tanh(x * 0.7978845608 * (1 + 0.044715 * x * x)))

In [5]:
parser = argparse.ArgumentParser(
    description='Instance Type Prediction using Ridle',
)
parser.add_argument('--dataset', nargs='?', default='Universities_DBpedia', type=str)
#parser.add_argument('--dataset', nargs='?', default='umls', type=str)
parser, unknown = parser.parse_known_args()

# Load Representations
print('Reading Data...')
df = pd.read_csv('./dataset/{}/embedding.csv'.format(parser.dataset))

# Load mapping
if 'dbp' in parser.dataset.lower():
    mapping = pd.read_json('./dataset/dbp_type_mapping.json')
elif 'wd' in parser.dataset.lower() or 'wikidata' in parser.dataset.lower():
    mapping = pd.read_json('./dataset/wd_mapping_type.json')
else:
    mapping = pd.read_json('./dataset/{}/type_mapping.json'.format(parser.dataset))

# merge them
print('Processing Data...')
r = pd.merge(df, mapping, on='S')

Reading Data...
Processing Data...


### Loading and reading the data

In [6]:
input_data = pd.read_pickle('./dataset/{}/input_data_unclean.pkl'.format(parser.dataset))
#input_data = pd.read_pickle('./dataset/{}/input_data_summary_full.pkl'.format(parser.dataset))

In [7]:
non_empty = input_data[input_data.Summary!='']

In [8]:
r_copy = r.copy()

In [9]:
r_new = r_copy[r_copy.S.isin(non_empty['S'].values)]

In [10]:
input_new = input_data[input_data.S.isin(non_empty['S'].values)]

In [11]:
text = input_new['Summary']
sentences = ["" + temp + "" for temp in text]

In [16]:
inputs = r_new.drop(['S', 'Class'], axis=1).values

#### Creating sentence embeddings and concatenated vectors
If you want to try out a different Sentece Embedding Model, you can change the model name in the cell below. For a list of all the pretrained models visit : https://www.sbert.net/docs/pretrained_models.html. The basic tests are run on the 'all-MiniLM-L6-v2' pretrained model, which is also the default model for the Sentence_Embedder.

In [12]:
from sentence_embeddings import Sentence_Embedder
#s = Sentence_Embedder(model_name="all-mpnet-base-v2")
s = Sentence_Embedder(model_name="all-MiniLM-L6-v2")
embeddings = s.embed(sentences)

In [13]:
embeddings = np.array(embeddings)
i = 0
dict_embeddings = {}
for entity in input_new["S"]:
    dict_embeddings[entity] = [embeddings[i]]
    i+=1

In [17]:
inputs = np.array(inputs)
ridle_dict = {}
i=0
for entity in r_new['S']:
    ridle_dict[entity] = [inputs[i]]
    i+=1

In [18]:
import numpy as np
ds = [ridle_dict, dict_embeddings]
d = {}
for k in ridle_dict.keys():
    d[k] = tuple(d[k] for d in ds)

In [19]:
conc_input =  []
for key in d:
    conc_input.append(np.concatenate((d[key][0][0], d[key][1][0])))
conc_input = np.array(conc_input)

In [33]:
emb_array = np.array(embeddings)
len(emb_array)

29897

In [17]:
len(conc_input)

54

# Concatenated Vectors

In [20]:
K_FOLD = 10
mlb = MultiLabelBinarizer()
fold_no = 1
loss_per_fold, f1_macro, f1_micro, f1_weighted = [], [], [], []
kfold = KFold(n_splits=K_FOLD, shuffle=True, random_state=42)
targets = mlb.fit_transform(r_new['Class'])
wrong_sbert = []
for train, test_new in kfold.split(conc_input, targets):

    model = Sequential()
    model.add(Dense(conc_input[train].shape[1], input_dim=conc_input[train].shape[1]))
    model.add(Activation(gelu, name='Gelu'))
    model.add(Dense(targets[train].shape[1], activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('Training...')
    history = model.fit(conc_input[train], targets[train], batch_size=64, validation_data=(conc_input[test_new], targets[test_new]), epochs=100)
    y_pred = model.predict(conc_input[test_new])
    y_pred[y_pred>=0.5]=1
    y_pred[y_pred<0.5]=0
    
    # Generate F1 scores
    scores = model.evaluate(conc_input[test_new], targets[test_new], verbose=0)
    f1_macro.append(f1_score(targets[test_new], y_pred, average='macro', zero_division=1))
    f1_micro.append(f1_score(targets[test_new], y_pred, average='micro', zero_division=1))
    f1_weighted.append(f1_score(targets[test_new], y_pred, average='weighted', zero_division=1))
    
    for m in range(len(test_new)):
        comp = targets[test_new][m] ==  y_pred[m]
        if not(comp.all()):
            #wrong_sbert.append((test_new[m], y_pred[m], targets[test_new][m]))
            wrong_sbert.append(test_new[m])

    

    print('Score for fold', fold_no, ':', model.metrics_names[0], 'of', scores[0], ';', 'F1-Macro:', f1_macro[-1], 'F1-Micro:', f1_micro[-1])
    loss_per_fold.append(scores[0])

    fold_no += 1
    
# Provide average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(loss_per_fold)):
    print('------------------------------------------------------------------------')
    print('> Fold', i+1, ' - Loss:', loss_per_fold[i], '- F1-Macro:', f1_macro[i], '%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print('> F1-Macro:', np.mean(f1_macro), '(+-', np.std(f1_macro), ')')
print('> F1-Micro:', np.mean(f1_micro), '(+-', np.std(f1_micro), ')')
print('> Loss:', np.mean(loss_per_fold))
print('------------------------------------------------------------------------')

# Save results to file
result = {}
f1_macro = np.array(f1_macro)
f1_micro = np.array(f1_micro)
f1_weighted = np.array(f1_weighted)
result['F1-Macro'] = np.mean(f1_macro)
result['F1-Macro_std'] = np.std(f1_macro)
result['F1-Micro'] = np.mean(f1_micro)
result['F1-Micro_std'] = np.std(f1_micro)
result['F1-Weighted'] = np.mean(f1_weighted)
result['F1-Weighted_std'] = np.std(f1_weighted)
result['Dataset'] = parser.dataset
result['method'] = 'Concatenated Vectors'
df_result = pd.DataFrame([result])
print(df_result)

if os.path.isfile('./evaluation_instance_type.csv'):
    df_result.to_csv('./evaluation_instance_type.csv', mode='a', header=False, index=False)
# else:
    df_result.to_csv('./evaluation_instance_type.csv', index=False)

model.summary()

Training...
Epoch 1/100
94/94 [==============================] - 1s 3ms/step - loss: 0.0556 - accuracy: 0.6443 - val_loss: 0.0136 - val_accuracy: 0.7910
Epoch 2/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0101 - accuracy: 0.6454 - val_loss: 0.0132 - val_accuracy: 0.5729
Epoch 3/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0098 - accuracy: 0.5415 - val_loss: 0.0134 - val_accuracy: 0.6165
Epoch 4/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0094 - accuracy: 0.5317 - val_loss: 0.0127 - val_accuracy: 0.5008
Epoch 5/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0088 - accuracy: 0.4637 - val_loss: 0.0132 - val_accuracy: 0.5293
Epoch 6/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0085 - accuracy: 0.5424 - val_loss: 0.0123 - val_accuracy: 0.5263
Epoch 7/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0081 - accuracy: 0.3995 - val_loss: 0.0120 - val_accuracy: 0.5173
E

Epoch 58/100
94/94 [==============================] - 0s 1ms/step - loss: 4.0936e-05 - accuracy: 0.5903 - val_loss: 0.0290 - val_accuracy: 0.5955
Epoch 59/100
94/94 [==============================] - 0s 1ms/step - loss: 4.6413e-05 - accuracy: 0.5977 - val_loss: 0.0295 - val_accuracy: 0.6090
Epoch 60/100
94/94 [==============================] - 0s 1ms/step - loss: 3.5224e-05 - accuracy: 0.5779 - val_loss: 0.0297 - val_accuracy: 0.5835
Epoch 61/100
94/94 [==============================] - 0s 1ms/step - loss: 3.3429e-05 - accuracy: 0.5522 - val_loss: 0.0298 - val_accuracy: 0.6331
Epoch 62/100
94/94 [==============================] - 0s 1ms/step - loss: 3.0542e-05 - accuracy: 0.5890 - val_loss: 0.0300 - val_accuracy: 0.5895
Epoch 63/100
94/94 [==============================] - 0s 1ms/step - loss: 2.9305e-05 - accuracy: 0.5637 - val_loss: 0.0304 - val_accuracy: 0.6241
Epoch 64/100
94/94 [==============================] - 0s 1ms/step - loss: 2.7689e-05 - accuracy: 0.5649 - val_loss: 0.0303 -

Epoch 14/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0051 - accuracy: 0.1171 - val_loss: 0.0070 - val_accuracy: 0.1278
Epoch 15/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0045 - accuracy: 0.1178 - val_loss: 0.0070 - val_accuracy: 0.2105
Epoch 16/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0040 - accuracy: 0.1221 - val_loss: 0.0076 - val_accuracy: 0.0647
Epoch 17/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 0.1056 - val_loss: 0.0080 - val_accuracy: 0.1880
Epoch 18/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0034 - accuracy: 0.1688 - val_loss: 0.0084 - val_accuracy: 0.0286
Epoch 19/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0030 - accuracy: 0.0979 - val_loss: 0.0079 - val_accuracy: 0.1549
Epoch 20/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0028 - accuracy: 0.0996 - val_loss: 0.0081 - val_accuracy: 0.1519
Epoch 

94/94 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 0.2399 - val_loss: 0.0413 - val_accuracy: 0.2316
Epoch 27/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 0.2732 - val_loss: 0.0431 - val_accuracy: 0.2481
Epoch 28/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0010 - accuracy: 0.2725 - val_loss: 0.0419 - val_accuracy: 0.2872
Epoch 29/100
94/94 [==============================] - 0s 1ms/step - loss: 9.5150e-04 - accuracy: 0.2623 - val_loss: 0.0446 - val_accuracy: 0.2015
Epoch 30/100
94/94 [==============================] - 0s 1ms/step - loss: 7.7687e-04 - accuracy: 0.2720 - val_loss: 0.0469 - val_accuracy: 0.1910
Epoch 31/100
94/94 [==============================] - 0s 1ms/step - loss: 7.5581e-04 - accuracy: 0.2211 - val_loss: 0.0454 - val_accuracy: 0.1789
Epoch 32/100
94/94 [==============================] - 0s 1ms/step - loss: 6.3955e-04 - accuracy: 0.2363 - val_loss: 0.0457 - val_accuracy: 0.2436
Epo

94/94 [==============================] - 0s 2ms/step - loss: 4.2673e-04 - accuracy: 0.6304 - val_loss: 0.0090 - val_accuracy: 0.6015
Epoch 39/100
94/94 [==============================] - 0s 2ms/step - loss: 3.4174e-04 - accuracy: 0.6378 - val_loss: 0.0092 - val_accuracy: 0.6120
Epoch 40/100
94/94 [==============================] - 0s 1ms/step - loss: 2.4873e-04 - accuracy: 0.6526 - val_loss: 0.0096 - val_accuracy: 0.5609
Epoch 41/100
94/94 [==============================] - 0s 1ms/step - loss: 3.0164e-04 - accuracy: 0.6204 - val_loss: 0.0095 - val_accuracy: 0.6992
Epoch 42/100
94/94 [==============================] - 0s 1ms/step - loss: 2.0710e-04 - accuracy: 0.6613 - val_loss: 0.0095 - val_accuracy: 0.5534
Epoch 43/100
94/94 [==============================] - 0s 2ms/step - loss: 1.7459e-04 - accuracy: 0.6045 - val_loss: 0.0096 - val_accuracy: 0.5714
Epoch 44/100
94/94 [==============================] - 0s 1ms/step - loss: 1.6318e-04 - accuracy: 0.6324 - val_loss: 0.0096 - val_accuracy

...

Epoch 18/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 0.7103 - val_loss: 0.0092 - val_accuracy: 0.8827
Epoch 19/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0034 - accuracy: 0.7841 - val_loss: 0.0093 - val_accuracy: 0.6812
Epoch 20/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0029 - accuracy: 0.7883 - val_loss: 0.0103 - val_accuracy: 0.9128
Epoch 21/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0029 - accuracy: 0.7145 - val_loss: 0.0094 - val_accuracy: 0.6015
Epoch 22/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0024 - accuracy: 0.7888 - val_loss: 0.0097 - val_accuracy: 0.6617
Epoch 23/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 0.8124 - val_loss: 0.0094 - val_accuracy: 0.8060
Epoch 24/100
94/94 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 0.7985 - val_loss: 0.0101 - val_accuracy: 0.8541
Epoch 

94/94 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 0.8859 - val_loss: 0.0241 - val_accuracy: 0.8917
Epoch 31/100
94/94 [==============================] - 0s 2ms/step - loss: 7.3397e-04 - accuracy: 0.8877 - val_loss: 0.0249 - val_accuracy: 0.8977
Epoch 32/100
94/94 [==============================] - 0s 2ms/step - loss: 6.2807e-04 - accuracy: 0.8832 - val_loss: 0.0255 - val_accuracy: 0.8767
Epoch 33/100
94/94 [==============================] - 0s 2ms/step - loss: 5.6379e-04 - accuracy: 0.8966 - val_loss: 0.0254 - val_accuracy: 0.8902
Epoch 34/100
94/94 [==============================] - 0s 2ms/step - loss: 4.6648e-04 - accuracy: 0.8767 - val_loss: 0.0261 - val_accuracy: 0.8737
Epoch 35/100
94/94 [==============================] - 0s 2ms/step - loss: 4.2671e-04 - accuracy: 0.8780 - val_loss: 0.0260 - val_accuracy: 0.8902
Epoch 36/100
94/94 [==============================] - 0s 2ms/step - loss: 3.7453e-04 - accuracy: 0.8647 - val_loss: 0.0269 - val_accuracy: 0.

94/94 [==============================] - 0s 2ms/step - loss: 1.8831e-04 - accuracy: 0.1233 - val_loss: 0.0092 - val_accuracy: 0.1308
Epoch 43/100
94/94 [==============================] - 0s 2ms/step - loss: 1.7025e-04 - accuracy: 0.1490 - val_loss: 0.0093 - val_accuracy: 0.1579
Epoch 44/100
94/94 [==============================] - 0s 2ms/step - loss: 1.2599e-04 - accuracy: 0.1519 - val_loss: 0.0101 - val_accuracy: 0.1173
Epoch 45/100
94/94 [==============================] - 0s 2ms/step - loss: 1.2710e-04 - accuracy: 0.1345 - val_loss: 0.0098 - val_accuracy: 0.1429
Epoch 46/100
94/94 [==============================] - 0s 2ms/step - loss: 1.0590e-04 - accuracy: 0.1587 - val_loss: 0.0099 - val_accuracy: 0.1669
Epoch 47/100
94/94 [==============================] - 0s 2ms/step - loss: 9.4521e-05 - accuracy: 0.1729 - val_loss: 0.0101 - val_accuracy: 0.1970
Epoch 48/100
94/94 [==============================] - 0s 2ms/step - loss: 9.5352e-05 - accuracy: 0.1664 - val_loss: 0.0101 - val_accuracy

94/94 [==============================] - 0s 2ms/step - loss: 4.4495e-05 - accuracy: 0.5116 - val_loss: 0.0291 - val_accuracy: 0.4887
Epoch 55/100
94/94 [==============================] - 0s 2ms/step - loss: 4.1128e-05 - accuracy: 0.4690 - val_loss: 0.0296 - val_accuracy: 0.4857
Epoch 56/100
94/94 [==============================] - 0s 2ms/step - loss: 3.8211e-05 - accuracy: 0.5001 - val_loss: 0.0293 - val_accuracy: 0.5218
Epoch 57/100
94/94 [==============================] - 0s 2ms/step - loss: 3.5630e-05 - accuracy: 0.4896 - val_loss: 0.0297 - val_accuracy: 0.5248
Epoch 58/100
94/94 [==============================] - 0s 2ms/step - loss: 3.3011e-05 - accuracy: 0.5018 - val_loss: 0.0300 - val_accuracy: 0.5188
Epoch 59/100
94/94 [==============================] - 0s 2ms/step - loss: 2.9827e-05 - accuracy: 0.4994 - val_loss: 0.0306 - val_accuracy: 0.4602
Epoch 60/100
94/94 [==============================] - 0s 2ms/step - loss: 2.8377e-05 - accuracy: 0.4576 - val_loss: 0.0302 - val_accuracy

   F1-Macro  F1-Macro_std  F1-Micro  F1-Micro_std  F1-Weighted  \
0  0.728721      0.174349  0.997724      0.001509     0.996935   

   F1-Weighted_std               Dataset                method  
0         0.002019  Universities_DBpedia  Concatenated Vectors  
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 434)               188790    
                                                                 
 Gelu (Activation)           (None, 434)               0         
                                                                 
 dense_19 (Dense)            (None, 12)                5220      
                                                                 
Total params: 194,010
Trainable params: 194,010
Non-trainable params: 0
_________________________________________________________________


In [348]:
wrong_id_ridle = []
for element in wrong_ridle:
    wrong_id_ridle.append(element[0])
    wrong_id_ridle.sort()

IndexError: invalid index to scalar variable.

In [ ]:
wrong_id_bert = []
for element in wrong_sbert:
    wrong_id_bert.append(element[0])
    wrong_id_bert.sort()

In [ ]:
pred_bert = []
for element in wrong_sbert:
    pred_bert.append(element[1])

In [291]:
targets_bert = []
for element in wrong_sbert:
    targets_bert.append(element[2])

In [292]:
pred_ridle = []
for element in wrong_ridle:
    pred_ridle.append(element[1])

In [293]:
targets_ridle = []
for element in wrong_ridle:
    targets_ridle.append(element[2])

In [23]:
df1 = pd.read_pickle("./Misclassification.pkl")

In [294]:
df1.at['{}'.format(parser.dataset), "Ridle misclassified"] = len(wrong_ridle)
df1.at['{}'.format(parser.dataset), "Sbert misclassified"] = len(wrong_sbert)
df1.at['{}'.format(parser.dataset), "Misclassified in both"] = set(wrong_id_ridle).intersection(wrong_id_bert)
df1.at['{}'.format(parser.dataset), "Prediction Bert"] = pred_bert
df1.at['{}'.format(parser.dataset), "Target Bert"] = targets_bert
df1.at['{}'.format(parser.dataset), "ID list Bert"] = wrong_id_bert
df1.at['{}'.format(parser.dataset), "Prediction Ridle"] = pred_ridle
df1.at['{}'.format(parser.dataset), "Target Ridle"] = targets_ridle
df1.at['{}'.format(parser.dataset), "ID list Ridle"] = wrong_id_ridle

In [350]:
df1

,0,Ridle misclassified,Sbert misclassified,Misclassified in both,ID list Bert,Prediction Bert,Target Bert,ID list Ridle,Prediction Ridle,Target Ridle,USE misclassified,ID list USE
Books_DBpedia,Books_DBpedia,22,23,"{4366, 6548, 5665, 6438, 1959, 1065, 7340, 939...","[311, 1065, 1401, 1959, 1985, 2047, 3337, 3441...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1065, 1401, 1959, 1985, 2047, 2925, 3441, 436...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...",21,"[600, 1142, 1492, 2071, 2099, 2162, 3593, 4559..."
ChemicalCompounds_DBpedia,ChemicalCompounds_DBpedia,6,8,"{5286, 2406, 7216, 1906, 4186, 3452}","[1906, 2406, 3452, 4186, 5286, 5323, 7216, 7929]","[[0.0, 1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 1.0, 0....","[[0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0...","[1906, 2406, 3452, 4186, 5286, 7216]","[[0.0, 1.0, 1.0, 0.0, 0.0], [0.0, 1.0, 1.0, 0....","[[0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1...",8,"[1928, 2428, 3476, 4212, 5314, 5351, 7248, 7963]"
Company_DBpedia,Company_DBpedia,142,133,"{3587, 7172, 4101, 7180, 3089, 6677, 4632, 259...","[133, 156, 182, 218, 489, 716, 818, 1068, 1114...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,...","[62, 119, 123, 133, 156, 182, 291, 489, 607, 6...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,...",242,"[2, 4, 6, 7, 8, 11, 30, 40, 108, 110, 154, 170..."
DBp_2016-04,DBp_2016-04,5959,4179,"{8193, 16389, 16391, 8201, 32788, 32789, 24602...","[646, 1282, 1414, 1603, 1703, 1736, 2247, 2269...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[646, 1282, 1414, 1599, 1603, 1655, 1682, 1703...",,,6419,"[63, 110, 128, 133, 153, 160, 227, 232, 320, 3..."
Person_DBpedia,Person_DBpedia,2831,2606,"{8192, 2, 24580, 4, 24585, 12, 19, 24606, 32, ...","[216, 261, 274, 278, 474, 852, 880, 963, 1087,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[216, 261, 274, 278, 775, 852, 880, 963, 1246,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",5533,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
Songs_DBpedia,Songs_DBpedia,151,181,"{0, 2560, 3073, 3588, 6, 1545, 3082, 3601, 361...","[0, 6, 9, 35, 80, 121, 142, 149, 168, 196, 199...","[[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]...","[[0, 0, 0, 0, 1, 0, 1, 0, 1], [0, 0, 0, 0, 1, ...","[0, 6, 80, 163, 168, 199, 223, 248, 260, 293, ...","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]...","[[0, 0, 0, 0, 1, 1, 0, 0, 1], [0, 0, 0, 0, 1, ...",281,"[0, 3, 9, 13, 15, 16, 22, 28, 44, 95, 107, 114..."
umls,umls,52,45,"{0, 1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...","[0, 1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...",48,"[9, 10, 17, 20, 22, 25, 26, 27, 34, 39, 40, 44..."
Universities_DBpedia,Universities_DBpedia,35,37,"{4362, 652, 1165, 4625, 4755, 3991, 3993, 4122...","[304, 462, 548, 652, 1095, 1120, 1133, 1165, 1...","[[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0,...","[[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, ...","[462, 548, 652, 843, 1015, 1095, 1120, 1133, 1...","[[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0,...","[[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, ...",39,"[526, 617, 726, 931, 1201, 1226, 1239, 1271, 1..."
Books_Dbpedia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [351]:
df1.to_pickle("./Misclassification.pkl")

Run the code below if you want to specify the dataset from Misclassification.pkl

In [33]:
pred = df1.iloc[2]['Prediction Bert']
targ = df1.iloc[2]['Target Bert']
ids = df1.iloc[2]['ID list Bert']
false_negative = {}
false_positive = {}
for i in range(len(pred)):
    for j in range(len(pred[i])):
        if targ[i][j] == 1 and pred[i][j] == 0:
            if mlb.classes_[j] in false_negative.keys():
                false_negative[mlb.classes_[j]].append(ids[i])
            else: 
                false_negative[mlb.classes_[j]] = [ids[i]] 
            
        elif targ[i][j] == 0 and pred[i][j] == 1:
            if mlb.classes_[j] in false_positive.keys():
                false_positive[mlb.classes_[j]].append(ids[i])
            else: 
                false_positive[mlb.classes_[j]] = [ids[i]] 


Run the code below, if you ran the Neural Network before

In [19]:
false_negative = {}
false_positive = {}
for i in range(len(pred_bert)):
    for j in range(len(pred_bert[i])):
        if targets_bert[i][j] == 1 and pred_bert[i][j] == 0:
            if mlb.classes_[j] in false_negative.keys():
                false_negative[mlb.classes_[j]].append(r['S'].iloc[wrong_id_bert[i]])
            else: 
                false_negative[mlb.classes_[j]] = [r['S'].iloc[wrong_id_bert[i]]] 
            
        elif targets_bert[i][j] == 0 and pred_bert[i][j] == 1:
            if mlb.classes_[j] in false_positive.keys():
                false_positive[mlb.classes_[j]].append(r['S'].iloc[wrong_id_bert[i]])
            else: 
                false_positive[mlb.classes_[j]] = [r['S'].iloc[wrong_id_bert[i]]]


In [20]:
fn = pd.DataFrame.from_dict(false_negative, orient='index')
fn.to_pickle("./false_negative/'{}'.pkl".format(parser.dataset))

In [21]:
fp = pd.DataFrame.from_dict(false_positive, orient='index')
fp.to_pickle("./false_positive/'{}'.pkl".format(parser.dataset))

# Only Sentence Embeddings

In [35]:
K_FOLD = 10
mlb = MultiLabelBinarizer()
fold_no = 1
loss_per_fold, f1_macro, f1_micro, f1_weighted = [], [], [], []
kfold = KFold(n_splits=K_FOLD, shuffle=True, random_state=42)
targets = mlb.fit_transform(input_new['Class'])
#emb_array = r.drop(['S', 'Class'], axis=1).values
for train, test_new in kfold.split(emb_array, targets):
    model = Sequential()
    model.add(Dense(emb_array[train].shape[1], input_dim=emb_array[train].shape[1]))
    model.add(Activation(gelu, name='Gelu'))
    model.add(Dense(targets[train].shape[1], activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('Training...')
    history = model.fit(emb_array[train], targets[train], batch_size=64, validation_data=(emb_array[test_new], targets[test_new]), epochs=100)
    y_pred = model.predict(emb_array[test_new])
    y_pred[y_pred>=0.5]=1
    y_pred[y_pred<0.5]=0
    
    # Generate F1 scores
    scores = model.evaluate(emb_array[test_new], targets[test_new], verbose=0)
    f1_macro.append(f1_score(targets[test_new], y_pred, average='macro', zero_division=1))
    f1_micro.append(f1_score(targets[test_new], y_pred, average='micro', zero_division=1))
    f1_weighted.append(f1_score(targets[test_new], y_pred, average='weighted', zero_division=1))

    print('Score for fold', fold_no, ':', model.metrics_names[0], 'of', scores[0], ';', 'F1-Macro:', f1_macro[-1], 'F1-Micro:', f1_micro[-1])
    loss_per_fold.append(scores[0])

    fold_no += 1
    
# Provide average scores
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(loss_per_fold)):
    print('------------------------------------------------------------------------')
    print('> Fold', i+1, ' - Loss:', loss_per_fold[i], '- F1-Macro:', f1_macro[i], '%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print('> F1-Macro:', np.mean(f1_macro), '(+-', np.std(f1_macro), ')')
print('> F1-Micro:', np.mean(f1_micro), '(+-', np.std(f1_micro), ')')
print('> Loss:', np.mean(loss_per_fold))
print('------------------------------------------------------------------------')

# Save results to file
result = {}
f1_macro = np.array(f1_macro)
f1_micro = np.array(f1_micro)
f1_weighted = np.array(f1_weighted)
result['F1-Macro'] = np.mean(f1_macro)
result['F1-Macro_std'] = np.std(f1_macro)
result['F1-Micro'] = np.mean(f1_micro)
result['F1-Micro_std'] = np.std(f1_micro)
result['F1-Weighted'] = np.mean(f1_weighted)
result['F1-Weighted_std'] = np.std(f1_weighted)
result['Dataset'] = parser.dataset
result['method'] = 'Only Embeddings'
df_result = pd.DataFrame([result])
print(df_result)

if os.path.isfile('./evaluation_instance_type.csv'):
    df_result.to_csv('./evaluation_instance_type.csv', mode='a', header=False, index=False)
# else:
    df_result.to_csv('./evaluation_instance_type.csv', index=False)

    
model.summary()

Training...
Epoch 1/100
421/421 [==============================] - 2s 3ms/step - loss: 0.0593 - accuracy: 0.9776 - val_loss: 0.0209 - val_accuracy: 0.9860
Epoch 2/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0168 - accuracy: 0.9845 - val_loss: 0.0143 - val_accuracy: 0.9883
Epoch 3/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0125 - accuracy: 0.8704 - val_loss: 0.0118 - val_accuracy: 0.9411
Epoch 4/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0107 - accuracy: 0.8006 - val_loss: 0.0107 - val_accuracy: 0.6622
Epoch 5/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0097 - accuracy: 0.4766 - val_loss: 0.0101 - val_accuracy: 0.9391
Epoch 6/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0089 - accuracy: 0.4879 - val_loss: 0.0097 - val_accuracy: 0.8023
Epoch 7/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0084 - accuracy: 0.5041 - val_loss: 0.0095 - val_accu

Epoch 12/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0067 - accuracy: 0.6558 - val_loss: 0.0085 - val_accuracy: 0.6428
Epoch 13/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0064 - accuracy: 0.7033 - val_loss: 0.0086 - val_accuracy: 0.6679
Epoch 14/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0062 - accuracy: 0.7009 - val_loss: 0.0087 - val_accuracy: 0.5064
Epoch 15/100
421/421 [==============================] - 1s 4ms/step - loss: 0.0060 - accuracy: 0.6875 - val_loss: 0.0086 - val_accuracy: 0.6468
Epoch 16/100
421/421 [==============================] - 2s 4ms/step - loss: 0.0058 - accuracy: 0.6282 - val_loss: 0.0086 - val_accuracy: 0.4876
Epoch 17/100
421/421 [==============================] - 2s 4ms/step - loss: 0.0056 - accuracy: 0.7290 - val_loss: 0.0087 - val_accuracy: 0.6900
Epoch 18/100
421/421 [==============================] - 2s 4ms/step - loss: 0.0054 - accuracy: 0.7343 - val_loss: 0.0087 - val_accuracy:

421/421 [==============================] - 1s 3ms/step - loss: 0.0044 - accuracy: 0.8172 - val_loss: 0.0084 - val_accuracy: 0.8097
Epoch 24/100
421/421 [==============================] - 1s 4ms/step - loss: 0.0042 - accuracy: 0.8498 - val_loss: 0.0085 - val_accuracy: 0.9274
Epoch 25/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0040 - accuracy: 0.8510 - val_loss: 0.0086 - val_accuracy: 0.9040
Epoch 26/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0038 - accuracy: 0.8625 - val_loss: 0.0087 - val_accuracy: 0.8819
Epoch 27/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0035 - accuracy: 0.8807 - val_loss: 0.0089 - val_accuracy: 0.8896
Epoch 28/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0034 - accuracy: 0.8863 - val_loss: 0.0090 - val_accuracy: 0.8809
Epoch 29/100
421/421 [==============================] - 1s 3ms/step - loss: 0.0031 - accuracy: 0.8811 - val_loss: 0.0092 - val_accuracy: 0.9171
Epoch

Epoch 79/100
421/421 [==============================] - 2s 4ms/step - loss: 3.7025e-05 - accuracy: 0.5515 - val_loss: 0.0220 - val_accuracy: 0.5408
Epoch 80/100
421/421 [==============================] - 2s 4ms/step - loss: 2.5886e-05 - accuracy: 0.5421 - val_loss: 0.0224 - val_accuracy: 0.5562
Epoch 81/100
421/421 [==============================] - 2s 4ms/step - loss: 2.6795e-05 - accuracy: 0.5410 - val_loss: 0.0226 - val_accuracy: 0.5411
Epoch 82/100
421/421 [==============================] - 2s 4ms/step - loss: 2.9883e-05 - accuracy: 0.5376 - val_loss: 0.0228 - val_accuracy: 0.5268
Epoch 83/100
421/421 [==============================] - 1s 4ms/step - loss: 3.2576e-05 - accuracy: 0.5246 - val_loss: 0.0228 - val_accuracy: 0.5375
Epoch 84/100
421/421 [==============================] - 1s 4ms/step - loss: 2.3169e-05 - accuracy: 0.5264 - val_loss: 0.0229 - val_accuracy: 0.5301
Epoch 85/100
421/421 [==============================] - 1s 4ms/step - loss: 1.8323e-05 - accuracy: 0.5277 - val_

421/421 [==============================] - 2s 4ms/step - loss: 1.4086e-05 - accuracy: 0.5733 - val_loss: 0.0281 - val_accuracy: 0.5532
Epoch 90/100
421/421 [==============================] - 2s 4ms/step - loss: 1.1648e-05 - accuracy: 0.5667 - val_loss: 0.0282 - val_accuracy: 0.5532
Epoch 91/100
421/421 [==============================] - 2s 4ms/step - loss: 1.2523e-05 - accuracy: 0.5635 - val_loss: 0.0283 - val_accuracy: 0.5455
Epoch 92/100
421/421 [==============================] - 2s 4ms/step - loss: 1.3196e-05 - accuracy: 0.5616 - val_loss: 0.0287 - val_accuracy: 0.5615
Epoch 93/100
421/421 [==============================] - 2s 4ms/step - loss: 1.2553e-05 - accuracy: 0.5570 - val_loss: 0.0286 - val_accuracy: 0.5488
Epoch 94/100
421/421 [==============================] - 2s 4ms/step - loss: 1.1328e-05 - accuracy: 0.5577 - val_loss: 0.0285 - val_accuracy: 0.5344
Epoch 95/100
421/421 [==============================] - 2s 4ms/step - loss: 9.2740e-06 - accuracy: 0.5526 - val_loss: 0.0288 

...

Epoch 11/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0070 - accuracy: 0.6275 - val_loss: 0.0084 - val_accuracy: 0.6191
Epoch 12/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0067 - accuracy: 0.6949 - val_loss: 0.0084 - val_accuracy: 0.4555
Epoch 13/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0065 - accuracy: 0.7281 - val_loss: 0.0083 - val_accuracy: 0.5247
Epoch 14/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0062 - accuracy: 0.7230 - val_loss: 0.0084 - val_accuracy: 0.7411
Epoch 15/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0060 - accuracy: 0.7073 - val_loss: 0.0084 - val_accuracy: 0.8809
Epoch 16/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0058 - accuracy: 0.8186 - val_loss: 0.0083 - val_accuracy: 0.5990
Epoch 17/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0056 - accuracy: 0.7509 - val_loss: 0.0083 - val_accuracy:

421/421 [==============================] - 2s 5ms/step - loss: 0.0047 - accuracy: 0.8782 - val_loss: 0.0083 - val_accuracy: 0.9568
Epoch 23/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0045 - accuracy: 0.9093 - val_loss: 0.0083 - val_accuracy: 0.9334
Epoch 24/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0043 - accuracy: 0.9183 - val_loss: 0.0084 - val_accuracy: 0.9197
Epoch 25/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0041 - accuracy: 0.9040 - val_loss: 0.0085 - val_accuracy: 0.9401
Epoch 26/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0039 - accuracy: 0.9265 - val_loss: 0.0085 - val_accuracy: 0.9334
Epoch 27/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0037 - accuracy: 0.9338 - val_loss: 0.0087 - val_accuracy: 0.9113
Epoch 28/100
421/421 [==============================] - 2s 5ms/step - loss: 0.0034 - accuracy: 0.9293 - val_loss: 0.0087 - val_accuracy: 0.9428
Epoch

Epoch 78/100
421/421 [==============================] - 2s 5ms/step - loss: 4.1869e-05 - accuracy: 0.6022 - val_loss: 0.0209 - val_accuracy: 0.6193
Epoch 79/100
421/421 [==============================] - 2s 5ms/step - loss: 3.2158e-05 - accuracy: 0.5812 - val_loss: 0.0209 - val_accuracy: 0.6005
Epoch 80/100
421/421 [==============================] - 2s 5ms/step - loss: 3.9311e-05 - accuracy: 0.5898 - val_loss: 0.0210 - val_accuracy: 0.5948
Epoch 81/100
421/421 [==============================] - 2s 5ms/step - loss: 2.9941e-05 - accuracy: 0.5738 - val_loss: 0.0210 - val_accuracy: 0.5905
Epoch 82/100
421/421 [==============================] - 2s 5ms/step - loss: 3.1128e-05 - accuracy: 0.5773 - val_loss: 0.0213 - val_accuracy: 0.5841
Epoch 83/100
421/421 [==============================] - 2s 5ms/step - loss: 2.7285e-05 - accuracy: 0.5735 - val_loss: 0.0216 - val_accuracy: 0.5868
Epoch 84/100
421/421 [==============================] - 2s 5ms/step - loss: 2.4305e-05 - accuracy: 0.5654 - val_

Epoch 89/100
421/421 [==============================] - 2s 5ms/step - loss: 1.4125e-05 - accuracy: 0.5307 - val_loss: 0.0221 - val_accuracy: 0.5323
Epoch 90/100
421/421 [==============================] - 2s 5ms/step - loss: 1.9498e-05 - accuracy: 0.5300 - val_loss: 0.0221 - val_accuracy: 0.5166
Epoch 91/100
421/421 [==============================] - 2s 5ms/step - loss: 1.4911e-05 - accuracy: 0.5220 - val_loss: 0.0223 - val_accuracy: 0.5376
Epoch 92/100
421/421 [==============================] - 2s 5ms/step - loss: 1.8420e-05 - accuracy: 0.5239 - val_loss: 0.0225 - val_accuracy: 0.5166
Epoch 93/100
421/421 [==============================] - 2s 5ms/step - loss: 1.5488e-05 - accuracy: 0.5212 - val_loss: 0.0224 - val_accuracy: 0.5152
Epoch 94/100
421/421 [==============================] - 2s 5ms/step - loss: 1.3240e-05 - accuracy: 0.5188 - val_loss: 0.0226 - val_accuracy: 0.5095
Epoch 95/100
421/421 [==============================] - 2s 4ms/step - loss: 1.9156e-05 - accuracy: 0.5153 - val_

Epoch 45/100
421/421 [==============================] - 2s 5ms/step - loss: 5.8704e-04 - accuracy: 0.8544 - val_loss: 0.0147 - val_accuracy: 0.8234
Epoch 46/100
421/421 [==============================] - 2s 5ms/step - loss: 5.2791e-04 - accuracy: 0.8389 - val_loss: 0.0148 - val_accuracy: 0.8264
Epoch 47/100
421/421 [==============================] - 2s 5ms/step - loss: 4.5091e-04 - accuracy: 0.8377 - val_loss: 0.0152 - val_accuracy: 0.8150
Epoch 48/100
421/421 [==============================] - 2s 5ms/step - loss: 3.8818e-04 - accuracy: 0.8262 - val_loss: 0.0155 - val_accuracy: 0.8394
Epoch 49/100
421/421 [==============================] - 2s 5ms/step - loss: 3.3520e-04 - accuracy: 0.8244 - val_loss: 0.0162 - val_accuracy: 0.8043
Epoch 50/100
421/421 [==============================] - 2s 5ms/step - loss: 2.9833e-04 - accuracy: 0.8196 - val_loss: 0.0164 - val_accuracy: 0.8361
Epoch 51/100
421/421 [==============================] - 2s 5ms/step - loss: 2.6014e-04 - accuracy: 0.8154 - val_